In [1]:
from poker import Range
from poker.hand import Combo
import holdem_calc

import holdem_functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
import dask
from dask import delayed
import multiprocessing as mp
import numpy as np

### Optimal bet using Kelly Criterion

In [2]:
def calc_kelly(win_odds, bet_odds):
    kelly_bet = round((bet_odds*win_odds-(1-win_odds))/bet_odds, 3)
    return kelly_bet

### Calculate the poker bet based on your hand, the board, and the villain range

In [3]:
def calc_poker_bet(my_hand, board, bankroll, return_multiplier, min_kelly = .5, recommended_kelly = .7, max_kelly = 1.2,
                   villain_range = Range('77+, AT+, KJ+'), calc_villain_range = True, use_dask = True, villain_hand = None,
                   exact_calculation = True, verbose = True, num_sims = 1, read_from_file = None, print_elapsed_time = True):
    
    """
    Output
    First element of odds contains my probability of winning/tie/lose 
    Second element of odds contains my probablities
    Third element of odds contains villain probabilities
    """
    
    # @param my_hand is my hand in the format of Combo('AdKs') - must be 2 cards
    # @param board is a list of cards in the format ['Kd', 'Ah', '2c']
    
    # @param  bankroll is my current bankroll 
    # @param return_multiplier is the kelly return multiplier (e.g. if you win 2:1 then return_multiplier is 2)
    # @params - exact_calculation, num_sims, read_from_file, villain_hand, verbose, print_elapsed_time will stay default
    # @param villain_range is the range of possible hands that the villain has - must be in the format Range('77+, AT+, KJ+')
    
    odds = holdem_calc.calculate_odds_villan(board = board, exact = exact_calculation, 
                                             num = num_sims, input_file = read_from_file, 
                                             hero_cards = my_hand, villan_cards = villain_hand, 
                                             verbose = verbose, print_elapsed_time = print_elapsed_time)
    
    if calc_villain_range == True:
        
        def calc_items(villain_combos=villain_range.combos):
    
            items = [holdem_calc.calculate_odds_villan(board = board, exact = exact_calculation, 
                                                       num = num_sims, input_file = read_from_file, 
                                                       hero_cards = my_hand, villan_cards = villain_hand, 
                                                       verbose = verbose, print_elapsed_time = print_elapsed_time) 
                     for villain_hand in villain_combos]
 
            return items
        
        ######### Calc items #########
        
        if use_dask == False:
            
            items = calc_items()
            
        else:
            
            num_cores = mp.cpu_count()
            delayed_list = []
            start_pos = 0
            end_pos = int(np.floor(len(villain_range.combos) / num_cores))
            chunk_len = int(np.floor(len(villain_range.combos) / num_cores))

            for chunk in range(num_cores):
                if chunk != num_cores - 1:
                    delayed_list.append(delayed(calc_items)(villain_range.combos[start_pos:end_pos]))
                    start_pos = start_pos + chunk_len
                    end_pos = end_pos + chunk_len

                else:
                    delayed_list.append(delayed(calc_items)(villain_range.combos[start_pos:]))
            
            items = list(np.concatenate(dask.compute(*delayed_list)))
            
        ######### End Calc items #########
        

        print(display(HTML(villain_range.to_html())))
        
#         for hand_ranking in holdem_functions.hand_rankings:
#             print(hand_ranking +": " + str(np.mean([res[1][1][hand_ranking] for res in items if res])))
        
        hr = [hand_ranking +": " + str(np.mean([res[1][1][hand_ranking] for res in items if res])) 
              for hand_ranking in holdem_functions.hand_rankings]
    
        range_odds = {}
        
        [range_odds.update({odd_type: np.mean([res[0][odd_type] for res in items if res])}) for odd_type in ["tie", "win", "lose"]]
     
        ######### Random Poker Hand Kelly Bet ############
        kelly_bet_random = calc_kelly(win_odds = odds[0]['win'], bet_odds = return_multiplier) # Assumes all players cards are random
        
        min_kelly_random = kelly_bet_random*min_kelly
        recommended_kelly_random = kelly_bet_random*recommended_kelly
        true_kelly_random = kelly_bet_random
        max_kelly_random = kelly_bet_random*max_kelly
         
        
        ######### Range-based Poker Hand Kelly Bet ############
        
        # kelly_bet_range assumes players cards are better than random - it tries to estimate the potential range 
        # of the villains cards since the villain did not fold (and likely continue to bet)
        kelly_bet_range = calc_kelly(win_odds = range_odds['win'], bet_odds = return_multiplier)
        min_kelly_range = kelly_bet_range*min_kelly
        recommended_kelly_range = kelly_bet_range*recommended_kelly
        true_kelly_range = kelly_bet_range
        max_kelly_range = kelly_bet_range*max_kelly
        
        
        kelly_recommendations = {'min_kelly_random':min_kelly_random*bankroll,
                                 'recommended_kelly_random':recommended_kelly_random*bankroll,
                                 'true_kelly_random':true_kelly_random*bankroll, 
                                 'max_kelly_random':max_kelly_random*bankroll,
                                 'min_kelly_range':min_kelly_range*bankroll,
                                 'recommended_kelly_range':recommended_kelly_range*bankroll,
                                 'true_kelly_range':true_kelly_range*bankroll, 
                                 'max_kelly_range':max_kelly_range*bankroll}
#         return {'kelly_recommendations':kelly_recommendations, 'odds':odds, 'hand_ranking':hr, 'items':items} # items too big
        return {'kelly_recommendations':kelly_recommendations, 'odds':odds, 'hand_ranking':hr}    
    else:    
        
        print('Warning - not considering range underestimates the players cards. Consider setting calc_villain_range to True')
        kelly_bet = calc_kelly(win_odds = odds[0]['win'], bet_odds = return_multiplier)
        min_kelly = kelly_bet*min_kelly
        true_kelly = kelly_bet
        max_kelly = kelly_bet*max_kelly
        
        kelly_recommendations = {'min_kelly':min_kelly*bankroll, 
                                 'true_kelly':true_kelly*bankroll,
                                 'max_kelly':max_kelly*bankroll}

        return {'kelly_recommendations':kelly_recommendations, 'odds':odds, 'hand_ranking':hr}

In [5]:
kelly_recommender = calc_poker_bet(Combo('AhAd'), board = ['Jd', '2h', '3s', '5s', '4h'], bankroll = 100, 
                                   use_dask = True, return_multiplier=2, calc_villain_range = True)

Time elapsed:  0.05681467056274414
Time elapsed: Time elapsed: Time elapsed:  0.0
Time elapsed:  0.003993511199951172
Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed:  0.0 0.003968954086303711
Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed:  Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed:  0.0
 0.0
Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed:  0.00.003993988037109375
Time elapsed:  
Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed:  0.00.0
Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed:  0.0

Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed:  0.0

Time elapsed:  0.0Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed: 
Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed:  0.0
Time elapsed:  0.0
Time el

AA,AKs,AQs,AJs,ATs,,,,,,,,
AKo,KK,KQs,KJs,,,,,,,,,
AQo,KQo,QQ,,,,,,,,,,
AJo,KJo,,JJ,,,,,,,,,
ATo,,,,TT,,,,,,,,
,,,,,99,,,,,,,
,,,,,,88,,,,,,
,,,,,,,77,,,,,
,,,,,,,,,,,,
,,,,,,,,,,,,
,,,,,,,,,,,,


None


In [6]:
kelly_recommender

{'kelly_recommendations': {'min_kelly_random': 31.15,
  'recommended_kelly_random': 43.61,
  'true_kelly_random': 62.3,
  'max_kelly_random': 74.75999999999999,
  'min_kelly_range': 26.3,
  'recommended_kelly_range': 36.82,
  'true_kelly_range': 52.6,
  'max_kelly_range': 63.12},
 'odds': [{'tie': 0.0797979797979798,
   'win': 0.7484848484848485,
   'lose': 0.1717171717171717},
  [{'High Card': 0.0,
    'Pair': 0.0,
    'Two Pair': 0.0,
    'Three of a Kind': 0.0,
    'Straight': 1.0,
    'Flush': 0.0,
    'Full House': 0.0,
    'Four of a Kind': 0.0,
    'Straight Flush': 0.0,
    'Royal Flush': 0.0},
   {'High Card': 0.24242424242424243,
    'Pair': 0.4,
    'Two Pair': 0.09090909090909091,
    'Three of a Kind': 0.015151515151515152,
    'Straight': 0.2515151515151515,
    'Flush': 0.0,
    'Full House': 0.0,
    'Four of a Kind': 0.0,
    'Straight Flush': 0.0,
    'Royal Flush': 0.0}]],
 'hand_ranking': ['High Card: 0.16326530612244897',
  'Pair: 0.4897959183673469',
  'Two Pair: 